# PVInspect integration with PyTorch

PVInspect interfaces directly with PyTorch, since PVInspect `ImageSequence`s can be simply converted into PyTorch datasets. To demonstrate this, let's first set up everything:

In [ ]:
!git clone https://github.com/ma0ho/pvinspect.git
%cd pvinspect
!git checkout rework
!pip install .

In [1]:
import pvinspect as pv
from pvinspect.data.image import DType
from pvinspect.integration.pytorch.dataset import ClassificationDataset
from torchvision.transforms import Compose, RandomHorizontalFlip, RandomVerticalFlip, ToTensor, ToPILImage

/usr/local/lib/python3.7/dist-packages/pvinspect/data/image/show_plugin.py:38: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Now, we load our sample dataset and only use the training data:

In [2]:
seq = pv.datasets.elpv().pandas.query("testset == False")
len(seq)

Unzipping...Done.


2324

Note that this has annotations available in form of boolean metra attributes:

In [3]:
seq[:3].meta

,original_filename,modality,defect_probability,wafer,crack,inactive,blob,finger,testset
0,cell0634.png,EL_IMAGE,1.0,poly,True,False,False,True,False
1,cell0805.png,EL_IMAGE,0.0,poly,False,False,False,False,False
2,cell2285.png,EL_IMAGE,0.0,poly,False,False,False,True,False


For training, we usually have some data augmentation pipeline. Let's set up a simple one:

In [4]:
tfms = Compose([ToPILImage(), RandomHorizontalFlip(), RandomVerticalFlip(), ToTensor()])

Now, converting `seq` into a PyTorch classification dataset using the transforms is as simple as:

In [5]:
ds = ClassificationDataset(seq, meta_classes=["crack", "inactive"], data_transform=tfms)
ds

With the `meta_classes` parameter, we specify the meta attributes that should be converted into one-hot class variables. We can now access individual elements from the dataset:

In [6]:
ds[0]

(tensor([[[0.2941, 0.2941, 0.2941,  ..., 0.2941, 0.2980, 0.2941],
          [0.2980, 0.2980, 0.2980,  ..., 0.2902, 0.2941, 0.2980],
          [0.3020, 0.3020, 0.3020,  ..., 0.2863, 0.2902, 0.2941],
          ...,
          [0.3098, 0.3098, 0.3137,  ..., 0.2824, 0.2824, 0.2784],
          [0.3098, 0.3098, 0.3098,  ..., 0.2824, 0.2784, 0.2784],
          [0.3098, 0.3137, 0.3137,  ..., 0.2784, 0.2784, 0.2784]]]),
 tensor([1., 0.]))

Here, we see that


1.   Image data is corretcly converted into PyTorch tensors, which shows that the transforms are applied
2.   Specified classes are converted into a one-hot vector and returned as a second return value

